In [125]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd
import sklearn
import numpy as np
import torch.optim as optim

In [30]:
def train_test_split(df, test_size=0.2):
  return sklearn.model_selection.train_test_split(df, test_size=test_size)

In [3]:
df = pd.read_csv('/content/Food_Supply_Quantity_kg_Data.csv')

In [9]:
clean_df = df.dropna()

In [41]:
clean_df.head(20)

,Country,Alcoholic Beverages,Animal fats,Animal Products,"Aquatic Products, Other",Cereals - Excluding Beer,Eggs,"Fish, Seafood",Fruits - Excluding Wine,Meat,Milk - Excluding Butter,Miscellaneous,Offals,Oilcrops,Pulses,Spices,Starchy Roots,Stimulants,Sugar & Sweeteners,Sugar Crops,Treenuts,Vegetable Oils,Vegetables,Vegetal Products,Obesity,Undernourished,Confirmed,Deaths,Recovered,Active,Population,Unit (all except Population)
0,Afghanistan,0.0014,0.1973,9.4341,0.0000,24.8097,0.2099,0.0350,5.3495,1.2020,7.5828,0.0728,0.2057,0.0700,0.2953,0.0574,0.8802,0.3078,1.3489,0.0000,0.0770,0.5345,6.7642,40.5645,4.5,29.8,0.142134,0.006186,0.123374,0.012574,38928000.0,%
1,Albania,1.6719,0.1357,18.7684,0.0000,5.7817,0.5815,0.2126,6.7861,1.8845,15.7213,0.1123,0.2324,0.9377,0.2380,0.0008,1.8096,0.1055,1.5367,0.0000,0.1515,0.3261,11.7753,31.2304,22.3,6.2,2.967301,0.050951,1.792636,1.123714,2838000.0,%
2,Algeria,0.2711,0.0282,9.6334,0.0000,13.6816,0.5277,0.2416,6.3801,1.1305,7.6189,0.1671,0.0870,0.3493,0.4783,0.0557,4.1340,0.2216,1.8342,0.0000,0.1152,1.0310,11.6484,40.3651,26.6,3.9,0.244897,0.006558,0.167572,0.070767,44357000.0,%
3,Angola,5.8087,0.0560,4.9278,0.0000,9.1085,0.0587,1.7707,6.0005,2.0571,0.8311,0.1165,0.1550,0.4186,0.6507,0.0009,18.1102,0.0508,1.8495,0.0000,0.0061,0.6463,2.3041,45.0722,6.8,25,0.061687,0.001461,0.056808,0.003419,32522000.0,%
5,Argentina,4.2672,0.2234,19.3454,0.0000,8.4102,0.9979,0.4693,6.0435,7.0421,10.2328,0.0000,0.3779,0.0116,0.0528,0.0122,3.0420,0.4378,3.0536,0.0000,0.0200,0.9541,4.3503,30.6559,28.5,4.6,4.356147,0.108227,3.905192,0.342729,45377000.0,%
6,Armenia,0.4014,0.1833,13.5640,0.0000,7.2982,0.5783,0.2896,6.0989,2.2675,9.9407,0.2355,0.3040,0.0899,0.1441,0.0055,2.0359,0.1863,2.6579,0.0000,0.1108,0.4705,16.7019,36.4358,20.9,4.3,5.681225,0.105345,5.398410,0.177470,2956000.0,%
7,Australia,5.5436,0.3143,21.4175,0.0033,5.4979,0.4428,1.4264,4.1883,6.7049,12.1018,0.5293,0.4240,0.3694,0.0546,0.0458,2.7884,0.2928,2.5364,0.0000,0.3176,1.2798,5.1406,28.5806,30.4,<2.5,0.112025,0.003530,0.101289,0.007207,25754000.0,%
8,Austria,7.0215,0.8555,19.5654,0.0011,6.2116,0.7884,0.7562,4.6069,4.6810,12.3776,0.0005,0.1052,0.2683,0.0456,0.0494,3.0548,0.4106,2.6094,0.0000,0.2367,0.8109,5.1098,30.4338,21.9,<2.5,4.739982,0.089679,4.496870,0.153433,8914000.0,%
9,Azerbaijan,3.5969,0.2544,11.6416,0.0000,13.0898,0.5593,0.2020,4.7988,2.1513,8.3212,0.0145,0.1534,0.0145,0.0347,0.0044,4.7041,0.0587,1.5523,0.0000,0.1906,0.2235,10.0755,38.3584,19.9,<2.5,2.285536,0.031223,2.225574,0.028740,10108000.0,%
11,Bangladesh,0.0031,0.0156,5.1926,0.0000,29.8045,0.2957,2.5221,2.8655,0.4191,1.8778,0.0197,0.0633,0.0820,0.7065,0.3195,5.3378,0.0529,0.9285,0.1950,0.0934,0.7615,3.6405,44.8033,3.4,14.7,0.316691,0.004823,0.284344,0.027524,169809000.0,%


In [45]:
avrages = clean_df.mean()
avrages[avrages > 3].astype(int)

Alcoholic Beverages                3
Animal Products                   12
Cereals - Excluding Beer          12
Fruits - Excluding Wine            5
Meat                               3
Milk - Excluding Butter            6
Starchy Roots                      5
Vegetables                         5
Vegetal Products                  37
Obesity                           18
Population                  47965785
dtype: int64

In [60]:
relevant_cols = ["Alcoholic Beverages", "Animal Products", "Cereals - Excluding Beer", "Fruits - Excluding Wine", "Milk - Excluding Butter", "Starchy Roots", "Vegetal Products", "Obesity"]
relevant_df = clean_df[relevant_cols]

In [61]:
column_renames = ["Alcohol", "Meat", "Cereals", "Fruits", "Milk", "Starchy Roots", "Vegetable", "Obesity"]
relevant_df.columns = column_renames

In [62]:
relevant_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 0 to 169
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Alcohol        154 non-null    float64
 1   Meat           154 non-null    float64
 2   Cereals        154 non-null    float64
 3   Fruits         154 non-null    float64
 4   Milk           154 non-null    float64
 5   Starchy Roots  154 non-null    float64
 6   Vegetable      154 non-null    float64
 7   Obesity        154 non-null    float64
dtypes: float64(8)
memory usage: 10.8 KB


In [58]:
avrages = relevant_df.mean()
avrages[avrages > 3].astype(int)

Alcohol           3
Meat             12
Cereals          12
Fruits            5
Milk              6
Starchy Roots     5
Vegetable        37
Obesety          18
dtype: int64

In [64]:
y_df = relevant_df["Obesity"]
X_df = relevant_df.drop(["Obesity"], axis=1)

In [139]:
y_values = np.array(y_df.values).astype(np.float32)
X_values = np.array(X_df.values).astype(np.float32)

In [149]:
y_tensors = torch.from_numpy(y_values).to(device)
X_tensors = torch.from_numpy(X_values).to(device)

In [192]:
y_train = y_tensors[10:]
y_test = y_tensors[:10]
X_train = X_tensors[10:]
X_test = X_tensors[:10]

In [193]:
X_values.shape

(154, 7)

In [194]:
fetech_req = [
    {
      "id": 5,
      "amount_g": 11000.0,
      "category": "Bagels and English muffins"
    },
    {
      "id": 10,
      "amount_g": 30.0,
      "category": "Candy containing chocolate"
    },
    {
      "id": 46,
      "amount_g": 102.600003,
      "category": "Chicken whole pieces"
    },
    {
      "id": 4,
      "amount_g": 4.0,
      "category": "Coffee"
    },
    {
      "id": 50,
      "amount_g": 525.599976,
      "category": "Cold cuts and cured meats"
    },
    {
      "id": 42,
      "amount_g": 100.0,
      "category": "Cream and cream substitutes"
    },
    {
      "id": 52,
      "amount_g": 262.799988,
      "category": "Not included in a food category"
    },
    {
      "id": 44,
      "amount_g": 78.780003,
      "category": "Other starchy vegetables"
    },
    {
      "id": 48,
      "amount_g": 929.599976,
      "category": "Rice mixed dishes"
    }
  ]

Model

In [195]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [196]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(7, 64),
        self.fc2 = nn.Linear(64, 64),
        self.fc3 = nn.Linear(64, 1),

    def forward(self, x):
        # x = self.flatten(x)
        x = F.relu(self.fc1)
        x = F.relu(self.fc2)
        x = self.fc3
        return F.log_softmax(x, dim=1)
net = NeuralNetwork().to(device)
net

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [197]:
logits = model(X_train)
len(logits)

torch.Size([144, 7])


144

In [200]:
net = NeuralNetwork().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss(reduction='sum')
EPOCHS = 3
for epoch in range(EPOCHS):
  for i in range(len(X_train)):
    X = X_train[i]
    y = y_train[i]
    net.zero_grad()
    output = net(X)
    loss = loss_fn(output, y)
    loss = F.l1_loss(output, y)
    loss.backward()
    optimizer.step()
  print(loss)


torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Si

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':


torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Si

In [201]:
ranges = []
total = 0

with torch.no_grad():
  for i in range(len(X_test)):
    X = X_test[i]
    y = y_test[i]
    output = net(X)
    ranges.append(abs(y-output[0]))
print(f"Avrage range: {sum(ranges)/len(ranges)}")

torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
Avrage range: 4.929542064666748


In [207]:
torch.save(net.state_dict(), "/content/model_state.pt")

In [208]:
torch.save(net, "/content/model.pt")

In [209]:
new_net = torch.load("/content/model.pt")

ranges = []
total = 0

with torch.no_grad():
  for i in range(len(X_test)):
    X = X_test[i]
    y = y_test[i]
    output = new_net(X)
    ranges.append(abs(y-output[0]))
print(f"Avrage range: {sum(ranges)/len(ranges)}")

torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
torch.Size([7])
Avrage range: 4.929542064666748
